# Imports

In [18]:
# <your imports>
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification
from transformers import EarlyStoppingCallback

from preprocessing import Preprocessing


# models
from models import BertClassifier

In [32]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Data loading/preprocessing

In [8]:
preprocessor = Preprocessing()

result = preprocessor.get_datasets()

train_set = result["train_set"]
val_set = result["val_set"]

# Model initialization

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
conf = OmegaConf.load("config.yaml").general

args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=1,
    per_device_train_batch_size=conf.batch_size,
    per_device_eval_batch_size=conf.batch_size,
    num_train_epochs=conf.epochs,
    seed=0,
    load_best_model_at_end=True,
)

In [33]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

## Train loop

In [34]:
# API key for wandb: 1fef7fd74ee7f9dfeb33b86d13b60f8ffe9c968a

In [ ]:
trainer.train()

# Custom Trainer

In [36]:
# from trainer import Trainer

In [ ]:
# preprocessor = Preprocessing()

# result = preprocessor.get_dataloaders()

# train_dl = result["train_dl"]
# val_dl = result["val_dl"]

In [ ]:
# trainer = Trainer()

In [ ]:
# trainer.train(train_dl, val_dl)

# Experiments